In [13]:
%%html
<div align="center"
     style="color: CornSilk;
            background-color: #2e2e2e;
            padding: 20px;
            border-radius: 10px;
            font-family: Arial, sans-serif;
            line-height: 1.6;">

  <h1 style="margin: 0;">Redes Neuronales y Aprendizaje Profundo</h1>

  <h3 style="margin-top: 10px; margin-bottom: 20px;">
    Máster Universitario en Inteligencia Artificial
  </h3>

  <p style="margin: 5px 0;">
    <strong>Universidad Internacional de La Rioja (UNIR)</strong>
  </p>
  <p style="margin: 5px 0;">
    <strong>Profesor:</strong> Pablo Negre
  </p>

  <hr style="border: 1px solid CornSilk; width: 80%; margin-top: 20px;">
</div>



In [9]:
# ===============================================
# Clasificador sencillo con TensorFlow/Keras
# ===============================================

import tensorflow as tf
from tensorflow.keras import layers, models

# 1. Cargar dataset MNIST directamente desde Keras
# Son imágenes de 28x28 píxeles de dígitos (0–9)
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

# 2. Preprocesamiento:
#   - Normalizamos los píxeles a [0,1] dividiendo entre 255
#   - "aplanamos" cada imagen de 28x28 = 784 valores
x_train = x_train.reshape(-1, 28*28).astype("float32") / 255.0
x_test = x_test.reshape(-1, 28*28).astype("float32") / 255.0

# 3. Definir el modelo secuencial con capas densas
#   - 2 capas ocultas con ReLU
#   - Capa de salida con softmax para clasificación multiclase (10 clases)
model = models.Sequential([
    layers.Dense(128, activation="relu", input_shape=(784,)),  # Capa oculta 1
    layers.Dense(64, activation="relu"),                       # Capa oculta 2
    layers.Dense(10, activation="softmax")                     # Capa de salida
])

# 4. Compilar el modelo
#   - Optimizador Adam
#   - Pérdida de entropía cruzada categórica (para clasificación multiclase)
#   - Métrica: exactitud
model.compile(optimizer="adam",
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])

# 5. Entrenamiento
model.fit(x_train, y_train, epochs=5, batch_size=32, validation_split=0.1)

# 6. Evaluación en el set de prueba
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=2)
print(f"Exactitud en test (TensorFlow): {test_acc:.4f}")


11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/5
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 10s 5ms/step - accuracy: 0.8693 - loss: 0.4452 - val_accuracy: 0.9693 - val_loss: 0.1054
Epoch 2/5
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 9s 5ms/step - accuracy: 0.9658 - loss: 0.1111 - val_accuracy: 0.9725 - val_loss: 0.0912
Epoch 3/5
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - accuracy: 0.9773 - loss: 0.0715 - val_accuracy: 0.9722 - val_loss: 0.0893
Epoch 4/5
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 8s 5ms/step - accuracy: 0.9843 - loss: 0.0514 - val_accuracy: 0.9747 - val_loss: 0.0854
Epoch 5/5
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 8s 5ms/step - accuracy: 0.9859 - loss: 0.0441 - val_accuracy: 0.9770 - val_loss: 0.0777
313/313 - 1s - 3ms/step - accuracy: 0.9765 - loss: 0.0860
Exactitud en test (TensorFlow): 0.9765


In [10]:
# ===============================================
# Clasificador sencillo con PyTorch
# ===============================================

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# 1. Transformaciones para el dataset
#   - Convertimos a tensor
#   - Normalizamos los valores de píxeles a [0,1]
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Lambda(lambda x: x.view(-1))  # Aplana la imagen de 28x28 -> vector de 784
])

# 2. Descargar y cargar dataset
train_dataset = datasets.MNIST(root="./data", train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root="./data", train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# 3. Definir la red neuronal
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(784, 128)   # Capa oculta 1
        self.fc2 = nn.Linear(128, 64)    # Capa oculta 2
        self.fc3 = nn.Linear(64, 10)     # Capa de salida

    def forward(self, x):
        x = torch.relu(self.fc1(x))      # ReLU en la primera capa
        x = torch.relu(self.fc2(x))      # ReLU en la segunda capa
        x = self.fc3(x)                  # Logits sin softmax (PyTorch lo aplica en la pérdida)
        return x

model = MLP()

# 4. Definir pérdida y optimizador
criterion = nn.CrossEntropyLoss()        # Entropía cruzada (ya incluye softmax implícito)
optimizer = optim.Adam(model.parameters())

# 5. Entrenamiento
for epoch in range(5):  # 5 épocas como en Keras
    model.train()
    for images, labels in train_loader:
        optimizer.zero_grad()            # Resetear gradientes
        outputs = model(images)          # Forward pass
        loss = criterion(outputs, labels)# Calcular pérdida
        loss.backward()                  # Backpropagation
        optimizer.step()                 # Actualizar pesos
    print(f"Epoch {epoch+1}: Loss = {loss.item():.4f}")

# 6. Evaluación en test
model.eval()
correct, total = 0, 0
with torch.no_grad():  # No calculamos gradientes en evaluación
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)  # Elegir la clase con mayor probabilidad
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Exactitud en test (PyTorch): {correct/total:.4f}")


100%|██████████| 9.91M/9.91M [00:00<00:00, 58.4MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 1.66MB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 14.1MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 6.52MB/s]


Epoch 1: Loss = 0.3008
Epoch 2: Loss = 0.0124
Epoch 3: Loss = 0.0427
Epoch 4: Loss = 0.0908
Epoch 5: Loss = 0.0168
Exactitud en test (PyTorch): 0.9750
